In [4]:
# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Other common imports
import numpy as np
import os
from pathlib import Path

In [2]:
#Tackling the SketchRNN Dataset

DOWNLOAD_ROOT = "http://download.tensorflow.org/data/"
FILENAME = "quickdraw_tutorial_dataset_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT + FILENAME,
                                cache_subdir="datasets/quickdraw",
                                extract=True)

1065304064/1065301781 [==============================] - 162s 0us/step


In [5]:
quickdraw_dir = Path(filepath).parent
train_files = sorted([str(path) for path in quickdraw_dir.glob("training.tfrecord-*")])
eval_files = sorted([str(path) for path in quickdraw_dir.glob("eval.tfrecord-*")])

In [6]:
train_files

['/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00000-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00001-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00002-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00003-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00004-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00005-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00006-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00007-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00008-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/training.tfrecord-00009-of-00010']

In [7]:
eval_files

['/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00000-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00001-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00002-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00003-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00004-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00005-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00006-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00007-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00008-of-00010',
 '/home/sfefilatyev/.keras/datasets/quickdraw/eval.tfrecord-00009-of-00010']

In [8]:
with open(quickdraw_dir / "eval.tfrecord.classes") as test_classes_file:
    test_classes = test_classes_file.readlines()
    
with open(quickdraw_dir / "training.tfrecord.classes") as train_classes_file:
    train_classes = train_classes_file.readlines()

In [9]:
assert train_classes == test_classes
class_names = [name.strip().lower() for name in train_classes]

In [10]:
sorted(class_names)

['aircraft carrier',
 'airplane',
 'alarm clock',
 'ambulance',
 'angel',
 'animal migration',
 'ant',
 'anvil',
 'apple',
 'arm',
 'asparagus',
 'axe',
 'backpack',
 'banana',
 'bandage',
 'barn',
 'baseball',
 'baseball bat',
 'basket',
 'basketball',
 'bat',
 'bathtub',
 'beach',
 'bear',
 'beard',
 'bed',
 'bee',
 'belt',
 'bench',
 'bicycle',
 'binoculars',
 'bird',
 'birthday cake',
 'blackberry',
 'blueberry',
 'book',
 'boomerang',
 'bottlecap',
 'bowtie',
 'bracelet',
 'brain',
 'bread',
 'bridge',
 'broccoli',
 'broom',
 'bucket',
 'bulldozer',
 'bus',
 'bush',
 'butterfly',
 'cactus',
 'cake',
 'calculator',
 'calendar',
 'camel',
 'camera',
 'camouflage',
 'campfire',
 'candle',
 'cannon',
 'canoe',
 'car',
 'carrot',
 'castle',
 'cat',
 'ceiling fan',
 'cell phone',
 'cello',
 'chair',
 'chandelier',
 'church',
 'circle',
 'clarinet',
 'clock',
 'cloud',
 'coffee cup',
 'compass',
 'computer',
 'cookie',
 'cooler',
 'couch',
 'cow',
 'crab',
 'crayon',
 'crocodile',
 'crow